In [2]:
import os
import cv2 as cv
import numpy as np

In [3]:
rgb_dir = './dataset/cropped/rgb'
depth_dir = './dataset/cropped/depth'
mask_rgb_dir = './dataset/masked_cropped/rgb'
masked_depth_path = 'dataset/masked_cropped/depth'

In [ ]:
def filter_noise_by_contours(image, min_area):
    # Find contours in the binary image
    contours, _ = cv.findContours(image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    # Create a blank image for the filtered contours
    filtered_mask= np.zeros_like(image)
    
    # Draw filtered contours on the blank image
    for contour in contours:
        area = cv.contourArea(contour)
        if area >= min_area:
            cv.drawContours(filtered_mask, [contour], -1, 255, thickness=cv.FILLED)

    filtered_image = cv.bitwise_and(image, filtered_mask)
    
    return filtered_image

In [ ]:
min_area = 1000

In [6]:
for file_name in os.listdir(rgb_dir):

    rgb_img_path = os.path.join(rgb_dir, file_name)
    depth_img_path = os.path.join(depth_dir, file_name)
    rgb_img = cv.imread(rgb_img_path)
    depth_img = cv.cvtColor(cv.imread(depth_img_path), cv.COLOR_BGR2GRAY)
    
    filtered_depth = filter_noise_by_contours(depth_img, min_area)
    
    _, mask = cv.threshold(depth_img, 100, 255, cv.THRESH_BINARY)
    
    kernel = np.ones((9, 9), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    masked_rgb = cv.bitwise_and(rgb_img, rgb_img, mask = mask.astype('uint8'))
        
    masked_rgb_path = os.path.join(mask_rgb_dir, file_name)
    cv.imwrite(masked_rgb_path, masked_rgb)
    cv.imwrite(masked_depth_path, filtered_depth)